In [ ]:
#!/usr/bin/env python

# Taken and modified from http://wiki.ros.org/rospy_tutorials/Tutorials/WritingImagePublisherSubscriber

# Python libs
import sys, time

# numpy and scipy
import numpy as np
from scipy.ndimage import filters

# OpenCV
import cv2

# Ros libraries
import roslib
import rospy

# Ros Messages
from sensor_msgs.msg import CompressedImage
# We do not use cv_bridge it does not support CompressedImage in python
# from cv_bridge import CvBridge, CvBridgeError

VERBOSE=True

class image_processing:

    def __init__(self):
        
        '''Initialize ros publisher, ros subscriber'''
        # topic where we publish
        #self.image_pub = rospy.Publisher("/output/image_raw/compressed",
        #    CompressedImage)
        #self.bridge = CvBridge()

        # subscribed Topic
        self.subscriber = rospy.Subscriber("/camera/rgb/image_raw/compressed",
            CompressedImage, self.callback,  queue_size = 1)
        if VERBOSE :
            print "subscribed to /camera/rgb/image_raw/compressed"


    def callback(self, ros_data):

        '''Callback function of subscribed topic. 
        Here images get converted and features detected'''
        if VERBOSE :
            print 'received image of type: "%s"' % ros_data.format

        #### direct conversion to CV2 ####
        np_arr = np.fromstring(ros_data.data, np.uint8)
        #image_np = cv2.imdecode(np_arr, cv2.CV_LOAD_IMAGE_COLOR)
        image_np = cv2.imdecode(np_arr, cv2.IMREAD_COLOR) # OpenCV >= 3.0:
        
        #### Feature detectors using CV2 #### 
        # "","Grid","Pyramid" + 
        # "FAST","GFTT","HARRIS","MSER","ORB","SIFT","STAR","SURF"
        #method = "GridFAST"
        #feat_det = cv2.FeatureDetector_create(method)
        #time1 = time.time()

        # convert np image to grayscale
        #featPoints = feat_det.detect(
        #    cv2.cvtColor(image_np, cv2.COLOR_BGR2GRAY))
        #time2 = time.time()
        #if VERBOSE :
        #    print '%s detector found: %s points in: %s sec.'%(method,
        #        len(featPoints),time2-time1)

        #for featpoint in featPoints:
        #    x,y = featpoint.pt
        #    cv2.circle(image_np,(int(x),int(y)), 3, (0,0,255), -1)
        
        cv2.imshow('cv_img', image_np)
        key = cv2.waitKey(1)

        self.stop_it = False
        if key == ord('q'):
            self.stop_it = True
            cv.destroyAllWindows()
            rospy.signal_shutdown('Quit')
            
        if VERBOSE:
            print("stop = ", self.stop_it)
            
        #### Create CompressedIamge ####
        #msg = CompressedImage()
        #msg.header.stamp = rospy.Time.now()
        #msg.format = "jpeg"
        #msg.data = np.array(cv2.imencode('.jpg', image_np)[1]).tostring()
        # Publish new image
        #self.image_pub.publish(msg)
        
        #self.subscriber.unregister()

def main(args):

    '''Initializes and cleanup ros node'''
    
    ic = image_processing()
    
    rospy.init_node('image_processing', anonymous=True)
    
    try:
        rospy.spin()
    except KeyboardInterrupt:
        print "Shutting down ROS image_processing module"
    
    cv2.destroyAllWindows()

if __name__ == '__main__':
    main(sys.argv)

subscribed to /camera/rgb/image_raw/compressed
received image of type: "rgb8; jpeg compressed bgr8"
('stop = ', False)
received image of type: "rgb8; jpeg compressed bgr8"
('stop = ', False)
received image of type: "rgb8; jpeg compressed bgr8"
('stop = ', False)
received image of type: "rgb8; jpeg compressed bgr8"
('stop = ', False)
received image of type: "rgb8; jpeg compressed bgr8"
('stop = ', False)
received image of type: "rgb8; jpeg compressed bgr8"
('stop = ', False)
received image of type: "rgb8; jpeg compressed bgr8"
('stop = ', False)
received image of type: "rgb8; jpeg compressed bgr8"
('stop = ', False)
received image of type: "rgb8; jpeg compressed bgr8"
('stop = ', False)
received image of type: "rgb8; jpeg compressed bgr8"
('stop = ', False)
received image of type: "rgb8; jpeg compressed bgr8"


/home/toto/miniconda2/envs/py27/lib/python2.7/site-packages/ipykernel_launcher.py:51: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead


('stop = ', False)
received image of type: "rgb8; jpeg compressed bgr8"
('stop = ', False)
received image of type: "rgb8; jpeg compressed bgr8"
('stop = ', False)
received image of type: "rgb8; jpeg compressed bgr8"
('stop = ', False)
received image of type: "rgb8; jpeg compressed bgr8"
('stop = ', False)
received image of type: "rgb8; jpeg compressed bgr8"
('stop = ', False)
received image of type: "rgb8; jpeg compressed bgr8"
('stop = ', False)
received image of type: "rgb8; jpeg compressed bgr8"
('stop = ', False)
received image of type: "rgb8; jpeg compressed bgr8"
('stop = ', False)
received image of type: "rgb8; jpeg compressed bgr8"
('stop = ', False)
received image of type: "rgb8; jpeg compressed bgr8"
('stop = ', False)
received image of type: "rgb8; jpeg compressed bgr8"
('stop = ', False)
received image of type: "rgb8; jpeg compressed bgr8"
('stop = ', False)
received image of type: "rgb8; jpeg compressed bgr8"
('stop = ', False)
received image of type: "rgb8; jpeg compresse

('stop = ', False)
received image of type: "rgb8; jpeg compressed bgr8"
('stop = ', False)
received image of type: "rgb8; jpeg compressed bgr8"
('stop = ', False)
received image of type: "rgb8; jpeg compressed bgr8"
('stop = ', False)
received image of type: "rgb8; jpeg compressed bgr8"
('stop = ', False)
received image of type: "rgb8; jpeg compressed bgr8"
('stop = ', False)
received image of type: "rgb8; jpeg compressed bgr8"
('stop = ', False)
received image of type: "rgb8; jpeg compressed bgr8"
('stop = ', False)
received image of type: "rgb8; jpeg compressed bgr8"
('stop = ', False)
received image of type: "rgb8; jpeg compressed bgr8"
('stop = ', False)
received image of type: "rgb8; jpeg compressed bgr8"
('stop = ', False)
received image of type: "rgb8; jpeg compressed bgr8"
('stop = ', False)
received image of type: "rgb8; jpeg compressed bgr8"
('stop = ', False)
received image of type: "rgb8; jpeg compressed bgr8"
('stop = ', False)
received image of type: "rgb8; jpeg compresse

received image of type: "rgb8; jpeg compressed bgr8"
('stop = ', False)
received image of type: "rgb8; jpeg compressed bgr8"
('stop = ', False)
received image of type: "rgb8; jpeg compressed bgr8"
('stop = ', False)
received image of type: "rgb8; jpeg compressed bgr8"
('stop = ', False)
received image of type: "rgb8; jpeg compressed bgr8"
('stop = ', False)
received image of type: "rgb8; jpeg compressed bgr8"
('stop = ', False)
received image of type: "rgb8; jpeg compressed bgr8"
('stop = ', False)
received image of type: "rgb8; jpeg compressed bgr8"
('stop = ', False)
received image of type: "rgb8; jpeg compressed bgr8"
('stop = ', False)
received image of type: "rgb8; jpeg compressed bgr8"
('stop = ', False)
received image of type: "rgb8; jpeg compressed bgr8"
('stop = ', False)
received image of type: "rgb8; jpeg compressed bgr8"
('stop = ', False)
received image of type: "rgb8; jpeg compressed bgr8"
('stop = ', False)
received image of type: "rgb8; jpeg compressed bgr8"
('stop = ',

...

received image of type: "rgb8; jpeg compressed bgr8"
('stop = ', False)
received image of type: "rgb8; jpeg compressed bgr8"
('stop = ', False)
received image of type: "rgb8; jpeg compressed bgr8"
('stop = ', False)
received image of type: "rgb8; jpeg compressed bgr8"
('stop = ', False)
received image of type: "rgb8; jpeg compressed bgr8"
('stop = ', False)
received image of type: "rgb8; jpeg compressed bgr8"
('stop = ', False)
received image of type: "rgb8; jpeg compressed bgr8"
('stop = ', False)
received image of type: "rgb8; jpeg compressed bgr8"
('stop = ', False)
received image of type: "rgb8; jpeg compressed bgr8"
('stop = ', False)
received image of type: "rgb8; jpeg compressed bgr8"
('stop = ', False)
received image of type: "rgb8; jpeg compressed bgr8"
('stop = ', False)
received image of type: "rgb8; jpeg compressed bgr8"
('stop = ', False)
received image of type: "rgb8; jpeg compressed bgr8"
('stop = ', False)
received image of type: "rgb8; jpeg compressed bgr8"
('stop = ',

[ERROR] [1596452825.115751, 15.658000]: bad callback: <bound method image_processing.callback of <__main__.image_processing instance at 0x7f2eafed8b40>>
Traceback (most recent call last):
  File "/opt/ros/melodic/lib/python2.7/dist-packages/rospy/topics.py", line 750, in _invoke_callback
    cb(msg)
  File "<ipython-input-1-bfee6af471e0>", line 80, in callback
    cv.destroyAllWindows()
NameError: global name 'cv' is not defined



('stop = ', False)
received image of type: "rgb8; jpeg compressed bgr8"
('stop = ', False)
received image of type: "rgb8; jpeg compressed bgr8"
('stop = ', False)
received image of type: "rgb8; jpeg compressed bgr8"
('stop = ', False)
received image of type: "rgb8; jpeg compressed bgr8"
('stop = ', False)
received image of type: "rgb8; jpeg compressed bgr8"
('stop = ', False)
received image of type: "rgb8; jpeg compressed bgr8"
('stop = ', False)
received image of type: "rgb8; jpeg compressed bgr8"
('stop = ', False)
received image of type: "rgb8; jpeg compressed bgr8"
('stop = ', False)
received image of type: "rgb8; jpeg compressed bgr8"
('stop = ', False)
received image of type: "rgb8; jpeg compressed bgr8"
('stop = ', False)
received image of type: "rgb8; jpeg compressed bgr8"
('stop = ', False)
received image of type: "rgb8; jpeg compressed bgr8"
('stop = ', False)
received image of type: "rgb8; jpeg compressed bgr8"
('stop = ', False)
received image of type: "rgb8; jpeg compresse

received image of type: "rgb8; jpeg compressed bgr8"
('stop = ', False)
received image of type: "rgb8; jpeg compressed bgr8"
('stop = ', False)
received image of type: "rgb8; jpeg compressed bgr8"
('stop = ', False)
received image of type: "rgb8; jpeg compressed bgr8"
('stop = ', False)
received image of type: "rgb8; jpeg compressed bgr8"
('stop = ', False)
received image of type: "rgb8; jpeg compressed bgr8"
('stop = ', False)
received image of type: "rgb8; jpeg compressed bgr8"
('stop = ', False)
received image of type: "rgb8; jpeg compressed bgr8"
('stop = ', False)
received image of type: "rgb8; jpeg compressed bgr8"
('stop = ', False)
received image of type: "rgb8; jpeg compressed bgr8"
('stop = ', False)
received image of type: "rgb8; jpeg compressed bgr8"
('stop = ', False)
received image of type: "rgb8; jpeg compressed bgr8"
('stop = ', False)
received image of type: "rgb8; jpeg compressed bgr8"
('stop = ', False)
received image of type: "rgb8; jpeg compressed bgr8"
('stop = ',

('stop = ', False)
received image of type: "rgb8; jpeg compressed bgr8"
('stop = ', False)
received image of type: "rgb8; jpeg compressed bgr8"
('stop = ', False)
received image of type: "rgb8; jpeg compressed bgr8"
('stop = ', False)
received image of type: "rgb8; jpeg compressed bgr8"
('stop = ', False)
received image of type: "rgb8; jpeg compressed bgr8"
('stop = ', False)
received image of type: "rgb8; jpeg compressed bgr8"
('stop = ', False)
received image of type: "rgb8; jpeg compressed bgr8"
('stop = ', False)
received image of type: "rgb8; jpeg compressed bgr8"
('stop = ', False)
received image of type: "rgb8; jpeg compressed bgr8"
('stop = ', False)
received image of type: "rgb8; jpeg compressed bgr8"
('stop = ', False)
received image of type: "rgb8; jpeg compressed bgr8"
('stop = ', False)
received image of type: "rgb8; jpeg compressed bgr8"
('stop = ', False)
received image of type: "rgb8; jpeg compressed bgr8"
('stop = ', False)
received image of type: "rgb8; jpeg compresse

In [7]:
cv2.destroyAllWindows()